In [1]:
# train 파일 불러오기
train = pd.read_csv("./raw/train.csv")

In [2]:
# key 파일 불러오기
key = pd.read_csv("./raw/key.csv")

In [3]:
# key 파일 store_nbr로 정렬
key = key.sort_values("store_nbr").set_index(np.arange(len(key)))

In [4]:
# train, key 파일 merge
target = train.merge(key, how="left", on = "store_nbr")

In [5]:
target.to_csv("target.csv", sep=",", index=False)

---

In [1]:
target = pd.read_csv("target.csv")

In [2]:
# units이 하나도 없는 item 찾기 - 없음
item_count = target.groupby("item_nbr")["units"].sum()
item_count[item_count.values == 0]

Series([], Name: units, dtype: int64)

In [3]:
# units이 하나도 없는 store 찾기 - 없음
store_count = target.groupby("store_nbr")["units"].sum()
store_count[store_count == 0]

Series([], Name: units, dtype: int64)

In [4]:
# units이 하나도 없는 station 찾기 - 없음
station_count = target.groupby("station_nbr")["units"].sum()
any(station_count == 0)

False

In [5]:
# unit의 종류
a = np.sort(target["units"].unique())
b = np.bincount(target["units"])

a

array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
         22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
         33,   34,   35,   36,   37,   38,   39,   40,   41,   42,   43,
         44,   45,   46,   47,   48,   49,   50,   51,   52,   53,   54,
         55,   56,   57,   58,   59,   60,   61,   62,   63,   64,   65,
         66,   67,   68,   69,   70,   71,   72,   73,   74,   75,   76,
         77,   78,   79,   80,   81,   82,   83,   84,   85,   86,   87,
         88,   89,   90,   91,   92,   93,   94,   95,   96,   97,   98,
         99,  100,  101,  102,  103,  104,  105,  106,  107,  108,  109,
        110,  111,  112,  113,  114,  115,  116,  117,  118,  119,  120,
        121,  122,  123,  124,  125,  126,  127,  128,  129,  130,  131,
        132,  133,  134,  135,  136,  137,  138,  139,  140,  141,  142,
        143,  144,  145,  146,  147,  148,  149,  1

In [6]:
# black friday 인 날만 보기
black_fridays = ['2012-11-20', '2012-11-21', '2012-11-22', '2012-11-23', '2012-11-24', '2012-11-25', '2012-11-26', '2013-11-26',
                '2013-11-27', '2013-11-28', '2013-11-29', '2013-11-30', '2013-12-01', '2013-12-02']
target_black = target[target["date"].isin(black_fridays)]

In [7]:
len(target[~target["date"].isin(black_fridays)]) + len(target[target["date"].isin(black_fridays)])

4617600

In [8]:
target_not_black = target[~target["date"].isin(black_fridays)]

In [9]:
target[(target["units"] == 5568) | (target["units"] == 3369) | (target["units"] == 577) | (target["units"] == 503)]

,date,store_nbr,item_nbr,units,station_nbr
1488312,2012-10-28,16,25,577,2
1580533,2012-11-15,37,5,5568,13
3190033,2013-11-21,37,5,3369,13
3260078,2013-12-08,17,9,503,20


In [10]:
# 그래프를 잘 보기 위해 unit이 이상하게 높은 3369, 5568 제거
target_rm = target[target["units"] < 3000]
len(target_rm)

4617598

In [ ]:
# item 별로 그래프 그리기
%matplotlib inline
def make_item_graph():
    for i in range(1, 2):
        item = target_rm[target_rm["item_nbr"] == i]
        item_black = target_black[target_black["item_nbr"] == i]
        plt.figure(figsize=(16, 4))
        plt.xlabel("item"+str(i))
        plt.bar(item["date"], item["units"])
        plt.bar(item_black["date"], item_black["units"], color="r")
        
make_item_graph()

In [42]:
# 20개 station에 대해서 item 별로 그래프 그리기
def make_station_graph():
    for i in range(1, 21):
        station = target_rm[target_rm["station_nbr"] == i]
        plt.figure(figsize=(16, 4))
        plt.xlabel("station"+str(i))
        plt.plot(pd.to_datetime(station["date"]), station["units"])
        

In [35]:
target.pivot_table(columns="store_nbr", index="item_nbr", values = "units", aggfunc=sum, margins=True, margins_name="All")

store_nbr,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,43,44,45,All
item_nbr,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,778
2,0,0,1083,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3269
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,675,0,0,675
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,348,348
5,0,55104,56663,0,0,0,62230,32647,45024,61866,...,50063,0,0,48454,0,35178,0,32834,0,846662
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,41398
7,0,0,0,0,0,0,721,0,0,0,...,0,0,0,0,0,0,0,0,0,721
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30345
9,27396,0,0,117123,0,50431,0,0,0,0,...,0,66679,5057,0,0,0,0,0,15833,916615


In [36]:
target.pivot_table(values="units", columns="station_nbr", index="item_nbr", aggfunc=sum, margins=True, margins_name="All")

station_nbr,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,All
item_nbr,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,778,0,0,778
2,0,0,0,0,0,0,1787,0,0,0,...,0,0,0,0,0,1482,0,0,0,3269
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,675
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,348,0,0,0,0,348
5,0,0,41641,32647,0,129471,161035,0,0,0,...,94700,163039,90282,0,0,133847,0,0,0,846662
6,0,0,0,0,0,0,0,0,41398,0,...,0,0,0,0,0,0,0,0,0,41398
7,0,0,0,0,0,721,0,0,0,0,...,0,0,0,0,0,0,0,0,0,721
8,0,0,30345,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30345
9,27396,0,101586,0,0,0,0,5057,117123,108635,...,0,140448,117110,0,27997,100022,35874,0,135367,916615
